In [1]:
!pip install finterstellar

In [2]:
# Import library
import finterstellar as fs
#System trading Library 설치: Finterstellar

import io, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
symbol_ = 'TSLA'
start_date_ = '2020-06-30'
end_date_ = '2025-06-30'
fee_ = 0.001

In [4]:
# 1. RSI Performance and evaluation

In [5]:
# finstaller.performance 를 dict 로 반환하는 함수

import re

def parse_performance_output(output_text):
    result = {}
    for line in output_text.strip().split("\n"):
        # 콜론으로 분리
        if ":" not in line:
            continue
        key, value = line.split(":", 1)
        key = key.strip()
        value = value.strip()

        # 퍼센트 처리
        if value.endswith("%"):
            num = float(value.strip("%")) / 100
            result[key] = num
        # 연도 처리
        elif value.endswith("yrs"):
            num = float(value.strip("yrs"))
            result[key] = num
        # 숫자 처리
        else:
            try:
                result[key] = float(value)
            except ValueError:
                result[key] = value  # 혹시 문자열이면 그대로 저장

    return result


def parse_performance_output(output_text, pre_text):
    result = {}
    for line in output_text.strip().split("\n"):
        # 콜론으로 분리
        if ":" not in line:
            continue
        key, value = line.split(":", 1)
        key = pre_text + "_" + key.strip()
        value = value.strip()

        # 퍼센트 처리
        if value.endswith("%"):
            num = float(value.strip("%")) / 100
            result[key] = num
        # 연도 처리
        elif value.endswith("yrs"):
            num = float(value.strip("yrs"))
            result[key] = num
        # 숫자 처리
        else:
            try:
                result[key] = float(value)
            except ValueError:
                result[key] = value  # 혹시 문자열이면 그대로 저장

    return result

In [6]:
# RSI 함수 정의
def get_rsi_performance(df, w_, buy_, sell_, fee) :
    fs.rsi(df, w=w_)
    fs.indicator_to_signal(df, factor='rsi', buy=buy_, sell=sell_)
    fs.position(df)
    fs.evaluate(df, cost= fee)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # 함수 실행 (print만 함)
        fs.performance(df, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    performance_dict = parse_performance_output(output_text, 'RSI')

    return performance_dict

In [7]:
# MACD 함수 정의
def get_macd_performance(df, buy_, sell_, fee) :
    fs.macd(df)
    fs.indicator_to_signal(df, factor='macd', buy=buy_, sell=sell_)
    fs.position(df)
    fs.evaluate(df, cost= fee)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # 함수 실행 (print만 함)
        fs.performance(df, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    performance_dict = parse_performance_output(output_text, 'MACD')

    return performance_dict

In [8]:
# Stochastics 함수 정의
# Stochastics 함수는 w_.값에 factor ('slow_k', 'indicator') 값을 이용한다
def get_stochastics_performance(symbol, df, factor_, buy_, sell_, fee_) :
    fs.stochastic(df, symbol, n=14, m=3, t=3)
    # indicator 값 추출을 위한 전처리 작업
    df['indicator'] = df['slow_k'] - df['slow_d']

    fs.indicator_to_signal(df, factor= factor_, buy=buy_, sell=sell_)
    fs.position(df)
    fs.evaluate(df, cost= fee_)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # 함수 실행 (print만 함)
        fs.performance(df, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    performance_dict = parse_performance_output(output_text, 'Stochastics')

    return performance_dict

In [9]:
# 기본 팩터자료 함수 정의
# symbol : 주식 약칭명
# start_date_ : 시작일자, end_date_ : 종료일자
# fee_ : 거래수수료
def get_rsi_performance_dataframe(symbol, start_date_, end_date_, fee_) :
    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    print(df.head())

    results = []
    # 최적의 w, buy_position, sell_position 을 찾기 위한 recursive
    for w_ in range(7, 23+1) :
        for buy_ in range(15, 45+1) :
            for sell_ in range(55, 85+1) :
                result_dict = {}

                # 파라미터 추가
                result_dict["symbol"] = symbol
                result_dict["w_param"] = w_
                result_dict["buy_param"] = buy_
                result_dict["sell_param"] = sell_

                perf = {}

                perf = get_rsi_performance(df, w_, buy_, sell_, fee_)

                result_dict.update(perf)

                # 리스트에 추가
                results.append(result_dict)

                print(f"[{len(results)}] {result_dict}")
            # 3중 Loop 로 인해 중간에 csv 저장
            pd.DataFrame(results).to_csv("get_rsi_performance.csv")

    df_result = pd.DataFrame(results)
    return df_result


In [10]:
from joblib import Parallel, delayed
import itertools

def get_rsi_list(symbol, df, w_, buy_, sell_, fee_) :
    result_dict = {}

    # 파라미터 추가
    result_dict["symbol"] = symbol
    result_dict["w_param"] = w_
    result_dict["buy_param"] = buy_
    result_dict["sell_param"] = sell_

    perf = {}
    perf = get_rsi_performance(df, w_, buy_, sell_, fee_)
    result_dict.update(perf)

    print(f"{result_dict}")

    return list(result_dict)

# get_rsi_performance_dataframe 개선 버전(Parallel)
def get_rsi_performance_dataframe_parallel(symbol, start_date_, end_date_, fee_) :
    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    print(df.head())

    results = []
    # 최적의 w, buy_position, sell_position 을 찾기 위한 recursive
    # Parallel 을 위해 w, buy, sell parameter 를 range 로 지저
    w_range = range(7, 23)
    buy_range = range(15,45)
    sell_range = range(55,85)

    # (w,buy,sell) 조합이 들어있는 list 생성
    param_combinations = list(itertools.product(w_range,buy_range, sell_range))

    # Parallel 사용
    # n_jobs = N (-1 : 모든 CPU 가용자원 사용)
    results = Parallel(n_jobs = -1, backend='threading', verbose=1)(
        delayed(get_rsi_list)(symbol, df, w, buy, sell, fee_)
                for(w, buy, sell) in param_combinations

    )

    df_result = pd.DataFrame(results)
    return df_result


In [11]:
# 기본 팩터자료 함수 정의
# symbol : 주식 약칭명
# start_date_ : 시작일자, end_date_ : 종료일자
# fee_ : 거래수수료
def get_macd_performance_dataframe(symbol, start_date_, end_date_, fee_) :
    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    print(df.head())

    results = []
    # 최적의 buy_position, sell_position 을 찾기 위한 recursive
    for buy_ in range(-10, 10+1) :
        for sell_ in range(-10, 10+1) :
            result_dict = {}

            # 파라미터 추가
            result_dict["symbol"] = symbol
            result_dict["buy_param"] = buy_
            result_dict["sell_param"] = sell_

            perf = {}

            perf = get_macd_performance(df, buy_, sell_, fee_)

            result_dict.update(perf)

            # 리스트에 추가
            results.append(result_dict)

            print(f"[{len(results)}] {result_dict}")

    df_result = pd.DataFrame(results)
    return df_result


In [12]:
# 기본 팩터자료 함수 정의
# symbol : 주식 약칭명
# start_date_ : 시작일자, end_date_ : 종료일자
# fee_ : 거래수수료
def get_stochastic_performance_dataframe(symbol, start_date_, end_date_, fee_) :
    df = fs.get_ohlc(symbol, start_date=start_date_, end_date=end_date_)
    print(df.head())

    results = []
    # 최적의 buy_position, sell_position 을 찾기 위한 recursive
    # 1. slow_k
    for buy_ in range(10, 40+1) :
        for sell_ in range(60, 90+1) :
            result_dict = {}

            # 파라미터 추가
            result_dict["symbol"] = symbol
            result_dict["factor"] = "slow_k"
            result_dict["buy_param"] = buy_
            result_dict["sell_param"] = sell_

            perf = {}

            perf = get_stochastics_performance(symbol, df, "slow_k", buy_, sell_, fee_)

            result_dict.update(perf)

            # 리스트에 추가
            results.append(result_dict)

            print(f"[{len(results)}] {result_dict}")


    # 2. indicator
    for buy_ in range(-10, 10+1) :
        for sell_ in range(-10, 10+1) :
            result_dict = {}

            # 파라미터 추가
            result_dict["symbol"] = symbol
            result_dict["factor"] = "indicator"
            result_dict["buy_param"] = buy_
            result_dict["sell_param"] = sell_

            perf = {}

            perf = get_stochastics_performance(symbol, df, "indicator", buy_, sell_, fee_)

            result_dict.update(perf)

            # 리스트에 추가
            results.append(result_dict)

            print(f"[{len(results)}] {result_dict}")

    df_result = pd.DataFrame(results)

    return df_result


In [ ]:
# df_rsi = get_rsi_performance_dataframe(symbol_, start_date_, end_date_, fee_)
# df_rsi

            TSLA
2020-06-30 71.99
2020-07-01 74.64
2020-07-02 80.58
2020-07-06 91.44
2020-07-07 92.66


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


{'symbol': 'TSLA', 'w_param': 7, 'buy_param': 15, 'sell_param': 55, 'RSI_CAGR': 0.04190000000000001, 'RSI_Accumulated return': 0.22649999999999998, 'RSI_Average return': 0.0321, 'RSI_Benchmark return': 2.4175, 'RSI_Number of trades': 7.0, 'RSI_Number of win': 4.0, 'RSI_Hit ratio': 0.5714, 'RSI_Investment period': 5.0, 'RSI_Sharpe ratio': 0.08, 'RSI_MDD': -0.24559999999999998, 'RSI_Benchmark MDD': -0.7363}


KeyError: "['signal_price'] not in index"

In [ ]:
# df_macd = get_macd_performance_dataframe(symbol_, start_date_, end_date_, fee_)
# df_macd

            TSLA
2020-06-30 71.99
2020-07-01 74.64
2020-07-02 80.58
2020-07-06 91.44
2020-07-07 92.66
[1] {'symbol': 'TSLA', 'buy_param': -10, 'sell_param': -10, 'MACD_CAGR': 0.253, 'MACD_Accumulated return': 2.0907, 'MACD_Average return': 0.0215, 'MACD_Benchmark return': 3.4126, 'MACD_Number of trades': 17.0, 'MACD_Number of win': 4.0, 'MACD_Hit ratio': 0.2353, 'MACD_Investment period': 5.0, 'MACD_Sharpe ratio': 0.35, 'MACD_MDD': -0.7128, 'MACD_Benchmark MDD': -0.7363}
[2] {'symbol': 'TSLA', 'buy_param': -10, 'sell_param': -9, 'MACD_CAGR': 0.0326, 'MACD_Accumulated return': 0.1739, 'MACD_Average return': 0.019, 'MACD_Benchmark return': 3.4126, 'MACD_Number of trades': 14.0, 'MACD_Number of win': 10.0, 'MACD_Hit ratio': 0.7143, 'MACD_Investment period': 5.0, 'MACD_Sharpe ratio': 0.03, 'MACD_MDD': -0.5092, 'MACD_Benchmark MDD': -0.7363}
[3] {'symbol': 'TSLA', 'buy_param': -10, 'sell_param': -8, 'MACD_CAGR': -0.02, 'MACD_Accumulated return': -0.0962, 'MACD_Average return': -0.0006, 'MACD

,symbol,buy_param,sell_param,MACD_CAGR,MACD_Accumulated return,MACD_Average return,MACD_Benchmark return,MACD_Number of trades,MACD_Number of win,MACD_Hit ratio,MACD_Investment period,MACD_Sharpe ratio,MACD_MDD,MACD_Benchmark MDD
0,TSLA,-10,-10,0.25,2.09,0.02,3.41,17.00,4.00,0.24,5.00,0.35,-0.71,-0.74
1,TSLA,-10,-9,0.03,0.17,0.02,3.41,14.00,10.00,0.71,5.00,0.03,-0.51,-0.74
2,TSLA,-10,-8,-0.02,-0.10,-0.00,3.41,11.00,7.00,0.64,5.00,-0.10,-0.52,-0.74
3,TSLA,-10,-7,-0.03,-0.15,-0.00,3.41,10.00,7.00,0.70,5.00,-0.12,-0.60,-0.74
4,TSLA,-10,-6,0.01,0.05,0.02,3.41,10.00,7.00,0.70,5.00,-0.03,-0.60,-0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,TSLA,10,6,0.05,0.29,0.04,3.41,11.00,4.00,0.36,5.00,0.09,-0.34,-0.74
437,TSLA,10,7,0.07,0.43,0.05,3.41,11.00,4.00,0.36,5.00,0.15,-0.34,-0.74
438,TSLA,10,8,0.05,0.27,0.04,3.41,11.00,5.00,0.45,5.00,0.08,-0.35,-0.74
439,TSLA,10,9,0.06,0.36,0.05,3.41,11.00,5.00,0.45,5.00,0.12,-0.35,-0.74


In [13]:
# df_sto = get_stochastic_performance_dataframe(symbol_, start_date_, end_date_, fee_)
# df_sto

In [14]:
def get_rsi_algorithm_info(symbol, start_date_, end_date_, w_, buy_, sell_, text, fee_) :
    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)

    # rsi
    fs.rsi(df, w=w_)
    fs.draw_chart(df, left='rsi', right= symbol)
    # 파일로 저장
    plt.savefig(f"{symbol}_price_rsi_{text}_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor='rsi', buy=buy_, sell=sell_)
    # Calc position
    fs.position(df)
    fs.draw_chart(df, left='rsi', right='position_chart')
    # 파일로 저장
    plt.savefig(f"{symbol}_rsi_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    # Evalution
    fs.evaluate(df, cost= fee_)
    fs.draw_chart(df, left='acc_rtn_dp', right=symbol)
    # 파일로 저장
    plt.savefig(f"{symbol}_rsi_{text}_evaluation_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    # get Performance
    fs.performance(df, rf_rate=0.02)

    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"{symbol}_rsi_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [15]:
def get_macd_algorithm_info(symbol, start_date_, end_date_, buy_, sell_,text,  fee_) :
    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)

    # macd
    fs.macd(df)
    fs.draw_chart(df, right=['macd','macd_signal','macd_oscillator'])
    # 파일로 저장
    plt.savefig(f"{symbol}_price_macd_{text}_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor='macd_oscillator', buy=buy_, sell=sell_)

    # Calc position
    fs.position(df)
    fs.draw_chart(df, right='position_chart', left='macd_oscillator')
    # 파일로 저장
    plt.savefig(f"{symbol}_macd_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()


    # Evalution
    fs.evaluate(df, cost=fee_)
    fs.performance(df, rf_rate=0.02)
    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"{symbol}_macd_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [16]:
def get_stochastic_algorithm_info(symbol, start_date_, end_date_, factor_, buy_, sell_,text, fee_) :
    df = fs.get_ohlc(symbol, start_date=start_date_, end_date=end_date_)

    # stochastic
    fs.stochastic(df, symbol, n=14, m=3, t=3)
    df['indicator'] = df['slow_k'] - df['slow_d']
    fs.draw_chart(df, left=factor_, right=symbol)
    # 파일로 저장
    plt.savefig(f"{symbol}_price_stochastic_{text}_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor=factor_, buy=buy_, sell=sell_)

    # Calc position
    fs.position(df)
    # fs.draw_chart(df, right='position_chart', left='macd_oscillator')
    # # 파일로 저장
    # plt.savefig(f"{symbol}_position_chart.png", dpi=300, bbox_inches="tight")
    # # 리소스 해제
    # plt.close()


    # Evalution
    fs.evaluate(df, cost=fee_)
    fs.performance(df, rf_rate=0.02)
    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"{symbol}_stochastic_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [17]:
# # Max CAGR
# df_max_cagr = df_rsi[df_rsi['RSI_CAGR'] == df_rsi['RSI_CAGR'].max()]
# df_max_cagr.iloc[0]

NameError: name 'df_rsi' is not defined

In [ ]:
# # Max Sharpe ratio
# df_max_sharpe_ratio = df_rsi[df_rsi['RSI_Sharpe ratio'] == df_rsi['RSI_Sharpe ratio'].max()]
# df_max_sharpe_ratio.iloc[0]

,5
symbol,TSLA
w_param,12
buy_param,15
sell_param,55
RSI_CAGR,0.08
RSI_Accumulated return,0.46
RSI_Average return,0.21
RSI_Benchmark return,1.56
RSI_Number of trades,2.00
RSI_Number of win,2.00


In [ ]:
# get_rsi_algorithm_info(df_max_cagr.iloc[0]['symbol'],start_date_,end_date_,df_max_cagr.iloc[0]['w_param'],df_max_cagr.iloc[0]['buy_param'],df_max_cagr.iloc[0]['sell_param'], fee_)

CAGR: 7.92%
Accumulated return: 45.90%
Average return: 21.24%
Benchmark return : 217.53%
Number of trades: 2
Number of win: 2
Hit ratio: 100.00%
Investment period: 5.0yrs
Sharpe ratio: 0.44
MDD: -12.24%
Benchmark MDD: -73.63%


In [18]:
symbol_list = ['^GSPC', '^KS11','MSFT', 'GOOG', 'AMZN', 'META', 'NVDA', 'TSLA', 'AAPL']

In [19]:
def process(symbol) :
    df_rsi = get_rsi_performance_dataframe(symbol, start_date_, end_date_, fee_)
    df_macd = get_macd_performance_dataframe(symbol, start_date_, end_date_, fee_)
    df_sto = get_stochastic_performance_dataframe(symbol, start_date_, end_date_, fee_)

    # write csv
    df_rsi.to_csv(f'{symbol}_rsi_{start_date_}_to_{end_date_}.csv',encoding='utf8')
    df_macd.to_csv(f'{symbol}_rsi_{start_date_}_to_{end_date_}.csv',encoding='utf8')
    df_sto.to_csv(f'{symbol}_rsi_{start_date_}_to_{end_date_}.csv',encoding='utf8')

    # find max value
    # RSI CAGR
    df_rsi_max_cagr = df_rsi[df_rsi['RSI_CAGR'] == df_rsi['RSI_CAGR'].max()].iloc[0]
    get_rsi_algorithm_info(df_rsi_max_cagr['symbol'],start_date_,end_date_,df_rsi_max_cagr['w_param'],df_rsi_max_cagr['buy_param'],df_rsi_max_cagr['sell_param'],'CAGR', fee_)

    # RSI Sharpe ratio
    df_rsi_max_sharpe_ratio = df_rsi[df_rsi['RSI_Sharpe ratio'] == df_rsi['RSI_Sharpe ratio'].max()].iloc[0]
    get_rsi_algorithm_info(df_rsi_max_sharpe_ratio['symbol'],start_date_,end_date_,df_rsi_max_sharpe_ratio['w_param'],df_rsi_max_sharpe_ratio['buy_param'],df_rsi_max_sharpe_ratio['sell_param'],'SharpeRatio', fee_)

    # MACD CAGR
    df_macd_max_cagr = df_macd[df_macd['MACD_CAGR'] == df_macd['MACD_CAGR'].max()].iloc[0]
    get_macd_algorithm_info(df_macd_max_cagr['symbol'],start_date_,end_date_,df_macd_max_cagr['buy_param'],df_macd_max_cagr['sell_param'],'CAGR', fee_)

    # MACD Sharpe ratio
    df_macd_max_sharpe_ratio = df_macd[df_macd['MACD_Sharpe ratio'] == df_macd['MACD_Sharpe ratio'].max()].iloc[0]
    get_macd_algorithm_info(df_macd_max_sharpe_ratio['symbol'],start_date_,end_date_,df_macd_max_sharpe_ratio['buy_param'],df_macd_max_sharpe_ratio['sell_param'],'SharpeRatio', fee_)

    # Stochastic CAGR
    df_sto_max_cagr = df_sto[df_sto['Stochastics_CAGR'] == df_sto['Stochastics_CAGR'].max()].iloc[0]
    get_stochastic_algorithm_info(df_sto_max_cagr['symbol'],start_date_,end_date_,df_sto_max_cagr['factor'],df_sto_max_cagr['buy_param'],df_sto_max_cagr['sell_param'],'CAGR', fee_)

    # Stochastic Sharpe ratio
    df_sto_max_sharpe_ratio = df_sto[df_sto['Stochastics_Sharpe ratio'] == df_sto['Stochastics_Sharpe ratio'].max()].iloc[0]
    get_stochastic_algorithm_info(df_sto_max_sharpe_ratio['symbol'],start_date_,end_date_,df_sto_max_sharpe_ratio['factor'],df_sto_max_sharpe_ratio['buy_param'],df_sto_max_sharpe_ratio['sell_param'],'SharpeRatio', fee_)


In [20]:
# for
for symbol in symbol_list :
    process(symbol)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[13259] {'symbol': 'AAPL', 'w_param': 20, 'buy_param': 39, 'sell_param': 76, 'RSI_CAGR': 0.0, 'RSI_Accumulated return': 0.0, 'RSI_Average return': nan, 'RSI_Benchmark return': 0.0203, 'RSI_Number of trades': 0.0, 'RSI_Number of win': 0.0, 'RSI_Hit ratio': 0.0, 'RSI_Investment period': 0.0, 'RSI_Sharpe ratio': 0.0, 'RSI_MDD': 0.0, 'RSI_Benchmark MDD': 0.0}
[13260] {'symbol': 'AAPL', 'w_param': 20, 'buy_param': 39, 'sell_param': 77, 'RSI_CAGR': 0.0, 'RSI_Accumulated return': 0.0, 'RSI_Average return': nan, 'RSI_Benchmark return': 0.0203, 'RSI_Number of trades': 0.0, 'RSI_Number of win': 0.0, 'RSI_Hit ratio': 0.0, 'RSI_Investment period': 0.0, 'RSI_Sharpe ratio': 0.0, 'RSI_MDD': 0.0, 'RSI_Benchmark MDD': 0.0}
[13261] {'symbol': 'AAPL', 'w_param': 20, 'buy_param': 39, 'sell_param': 78, 'RSI_CAGR': 0.0, 'RSI_Accumulated return': 0.0, 'RSI_Average return': nan, 'RSI_Benchmark return': 0.0203, 'RSI_Number of trades': 0.0, 'RSI_Number of win': 0.0, 'RSI_Hit 